In [1]:
import findspark
findspark.init('/home/gani/spark-2.4.4-bin-hadoop2.7')
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('CodeAlong').getOrCreate()

In [3]:
df = spark.read.csv('titanic.csv',inferSchema=True,header=True)

In [4]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [6]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [7]:
cols = df.select(['Survived',
 'Pclass','Sex',
 'Age',
 'SibSp',
 'Parch','Fare','Embarked'])

In [8]:
finalDF = cols.na.drop()

In [9]:
from pyspark.ml.feature import VectorAssembler,VectorIndexer,OneHotEncoder, StringIndexer

In [10]:
genderIndexer = StringIndexer(inputCol='Sex',outputCol='SexIndexer')
genderEncoder = OneHotEncoder(inputCol='SexIndexer',outputCol='SexVec')

In [11]:
embarkIndexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkedIndexer')
embarkedEncoder = OneHotEncoder(inputCol='EmbarkedIndexer',outputCol='EmbarkedVec')

In [12]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec', 'EmbarkedVec','Age','SibSp','Parch','Fare'], 
                                      outputCol='features')

In [13]:
from pyspark.ml.classification import LogisticRegression

In [16]:
from pyspark.ml import Pipeline

In [15]:
logRegTitanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [17]:
pipeline = Pipeline(stages = [genderIndexer, embarkIndexer,
                             genderEncoder, embarkedEncoder,
                             assembler, logRegTitanic])

In [18]:
trainData, testData = finalDF.randomSplit([0.7,0.3])

In [19]:
fitModel = pipeline.fit(trainData)

In [20]:
results = fitModel.transform(testData)

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [22]:
binEval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [23]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [24]:
AUC = binEval.evaluate(results)

In [25]:
AUC

0.7906131479140328